[View in Colaboratory](https://colab.research.google.com/github/arash-ash/pRFsim/blob/master/pRFsim_Tutorial.ipynb)

In [17]:
!pip install git+https://github.com/arash-ash/pRFsim.git

  Cloning https://github.com/arash-ash/pRFsim.git to /tmp/pip-req-build-xs1m2w8c
  Running setup.py bdist_wheel for prfsim ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-_s3q31ww/wheels/12/53/5c/4d216ea97633dc7e012fb22b97224556339290b18fc5b5f756
Successfully built prfsim
  Found existing installation: prfsim 0.1.10
    Uninstalling prfsim-0.1.10:
      Successfully uninstalled prfsim-0.1.10


In [0]:
import prfsim.sim as psim
import numpy as np

In [0]:
#@experiment parameters
radius = 10
precision = 0.1 #@param {type:"slider", min:0, max:1.0, step:0.01}
barWidth = radius / 4
angles = [-90, 45, -180, 315, 90, 225, 0, 135]
nFrames = len(angles)*3
TR = 3.0
TRs = 5 # number of TRs for each frame
duration = nFrames*TRs

x, y = np.mgrid[-radius:radius:precision,
                -radius:radius:precision]
pos = np.dstack((x, y))
length = len(x[0])
nVoxels = 6 #@param {type:"slider", min:0, max:100, step:1}

# parameters for double gamma distribution function hrf:
n1 = 4
lmbd1 = 2.0
t01 = 0
n2 = 7
lmbd2 = 3
t02 = 0
a = 0.3

t = np.arange(0,nFrames*TRs*TR,TR)

In [0]:
hrf_gen = psim.hrf_double_gamma(t, n1, n2, lmbd1, lmbd2, t01, t02, a)
print('HRF generated')
hrf_est = hrf_gen

stim = psim.generateStim(radius=radius, precision=precision,
                    barWidth=barWidth, angles=angles,
                    nFrames=nFrames, length=length,
		                TR=TR, TRs=TRs, isCheckerboard=False)
print('stimulus generated')


neuronal_responses = psim.getNeuronalResponse(stim=stim, nVoxels=nVoxels,
                                        radius=radius, precision=precision,
                                        duration=duration)
print('Neuronal responses generated')


bolds = psim.generateData(neuronal_responses=neuronal_responses,
                     hrf=hrf_gen,
                     duration=duration, nVoxels=nVoxels)
print('BOLD responses generated')


print('pRF estimations started...')
results = psim.estimateAll(bolds=bolds, stim=stim,
                      hrf=hrf_est, radius=radius,
                      precision=precision,
                      nVoxels=nVoxels, margin = 1)
print('pRF estimation errors generated')